In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import os
import pickle

from keras import optimizer_v1
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, regularizers, layers, backend
from PIL import Image

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
    tpu = None

# TPUStrategy for distributed training
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else: # default strategy that works on CPU and single GPU
    strategy = tf.distribute.get_strategy()


In [ ]:
flowers = ["bougainvillea", "daisy", "dandelion", "gardenia", "hibiscus", "hydrangea",
          "iris", "lily", "lotus", "morningglory", "peachflower","peony", "phalaenopsis",
           "rose", "sunflower", "tulip"]

root_path = '../input/flowers-16classes/'

def readImg(root_path):
    x = []
    y = []
    for i, flower in enumerate(flowers):
        train_folder_path = root_path + flower  # 通过字典拼接文件夹路径
        for root, dirs, files in os.walk(train_folder_path):
            for file in files:
                img_path = train_folder_path + "/" + file  # 拼接图片路径
                img = Image.open(img_path)  # 读入图片
                img = img.convert("RGB")
                img = img.resize((224, 224), Image.ANTIALIAS)
                img_arr = np.array(img)  # 转为rgb图像数组
                img_arr = img_arr / 255.  # 数据归一化
                x.append(img_arr.astype(np.float32))
                y.append(i)
                print("loading: " + file)
    x = np.array(x)
    y = np.array(y, dtype=np.int64)
    return x, y, x.shape[0]

def load_data():
    print('-------------Generate Datasets-----------------')
    x, y, xlen = readImg(root_path)
    print("x: ", x.shape, "y: ", y.shape)
    np.random.seed(116)  # 使用相同的seed，保证输入特征和标签一一对应
    np.random.shuffle(x)
    np.random.seed(116)
    np.random.shuffle(y)
    return x, y

x, y = load_data()
split1 = int(0.3 * x.shape[0])
x_val_test, y_val_test = x[:split1], y[:split1]
x_train, y_train = x[split1:], y[split1:]

split2 = int(0.5 * x_val_test.shape[0])
x_test, y_test = x_val_test[:split2], y_val_test[:split2]
x_validation, y_validation = x_val_test[split2:], y_val_test[split2:]
print("x_train: {}, y_train: {}".format(x_train.shape, y_train.shape))
print("x_test: {}, y_test: {}".format(x_test.shape, y_test.shape))
print("x_validation: {}, y_validation: {}".format(x_validation.shape, y_validation.shape))

# AlexNet

In [ ]:
class AlexNet8(Model):
    def __init__(self):
        super(AlexNet8, self).__init__()
        self.c1 = Conv2D(filters=96, kernel_size=(3, 3))
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.p1 = MaxPool2D(pool_size=(3, 3), strides=2)

        self.c2 = Conv2D(filters=256, kernel_size=(3, 3))
        self.b2 = BatchNormalization()
        self.a2 = Activation('relu')
        self.p2 = MaxPool2D(pool_size=(3, 3), strides=2)

        self.c3 = Conv2D(filters=384, kernel_size=(3, 3), padding='same',
                         activation='relu')
                         
        self.c4 = Conv2D(filters=384, kernel_size=(3, 3), padding='same',
                         activation='relu')
                         
        self.c5 = Conv2D(filters=256, kernel_size=(3, 3), padding='same',
                         activation='relu')
        self.p3 = MaxPool2D(pool_size=(3, 3), strides=2)

        self.flatten = Flatten()
        self.f1 = Dense(2048, activation='relu')
        self.d1 = Dropout(0.5)
        self.f2 = Dense(2048, activation='relu')
        self.d2 = Dropout(0.5)
        self.f3 = Dense(16, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)

        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p2(x)

        x = self.c3(x)

        x = self.c4(x)

        x = self.c5(x)
        x = self.p3(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.d1(x)
        x = self.f2(x)
        x = self.d2(x)
        y = self.f3(x)
        return y
    
    

# 迁移学习
## ResNet50
ResNet50(weights='imagenet', include_top=False, pooling="avg") \
Dense(16, activation="softmax")

In [ ]:
class ResNet152(Model):
    def __init__(self):
        super(ResNet152, self).__init__()
        self.net = tf.keras.applications.ResNet152(
            weights="imagenet",
            include_top=False,
            pooling="avg"
        )
        self.net.trainable = False
        self.f = Dense(16, activation="softmax") 

    def call(self, x):
        x = self.net(x)
        y = self.f(x)
        return y

# 迁移学习
## VGG19
VGG19(weights='imagenet', include_top=False, pooling="max) \
Dense(1024, activation="relu") \
BatchNormalization() \
Dropout(0.5) \
Dense(16, activation="softmax")

In [ ]:
class VGG19(Model):
    def __init__(self):
        super(VGG19, self).__init__()
        self.net = tf.keras.applications.VGG19(
            weights="imagenet",
            include_top=False,
            pooling="max"
        )
        self.net.trainable = False
        self.f1 = Dense(1024, activation="relu")
        self.b1 = BatchNormalization()
        self.d1 = Dropout(0.5)
        self.f2 = Dense(16, activation="softmax") 

    def call(self, x):
        x = self.net(x)
        x = self.f1(x)
        x = self.b1(x)
        x = self.d1(x)
        y = self.f2(x)
        return y

# 迁移学习
## InceptionV3
InceptionV3(weights='imagenet', include_top=False, pooling="max) \
Dense(1024, activation="relu") \
BatchNormalization() \
Dropout(0.5) \
Dense(16, activation="softmax")

In [ ]:
class InceptionV3(Model):
    def __init__(self):
        super(InceptionV3, self).__init__()
        self.net = tf.keras.applications.InceptionV3(
            weights="imagenet",
            include_top=False,
            pooling="max"
        )
        self.net.trainable = False
        self.f1 = Dense(1024, activation="relu")
        self.b1 = BatchNormalization()
        self.d1 = Dropout(0.5)
        self.f2 = Dense(16, activation="softmax") 

    def call(self, x):
        x = self.net(x)
        x = self.f1(x)
        x = self.b1(x)
        x = self.d1(x)
        y = self.f2(x)
        return y

# 迁移学习
## DenseNet121
DenseNet121(weights='imagenet', include_top=False, pooling="max") \
Dense(1024, activation="relu") \
BatchNormalization() \
Dropout(0.5) \
Dense(16, activation="softmax")

In [ ]:
class DenseNet121(Model):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.net = tf.keras.applications.DenseNet121(
            weights="imagenet",
            include_top=False,
            pooling="max"
        )
        self.net.trainable = False
        self.f1 = Dense(1024, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())
        self.b1 = BatchNormalization()
        self.d1 = Dropout(0.5)
        self.f2 = Dense(16, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2()) 

    def call(self, x):
        x = self.net(x)
        x = self.f1(x)
        x = self.b1(x)
        x = self.d1(x)
        y = self.f2(x)
        return y

In [ ]:
class MobileNetV2(Model):
    def __init__(self):
        super(MobileNetV2, self).__init__()
        self.net = tf.keras.applications.MobileNetV2(
            input_shape=(224, 224, 3),
            weights="imagenet",
            include_top=False,
            pooling="avg"
        )
        self.net.trainable = False
        self.f1 = Dense(16, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2()) 

    def call(self, x):
        x = self.net(x)
        y = self.f1(x)
        return y

# MyModel & ResNet18

In [ ]:
class ResNet18(Model):

    def __init__(self, block_list, initial_filters=64):  # block_list表示每个block有几个卷积层
        super(ResNet18, self).__init__()
        self.num_blocks = len(block_list)  # 共有几个block
        self.block_list = block_list
        self.out_filters = initial_filters
        self.c1 = Conv2D(self.out_filters, (3, 3), strides=1, padding='same', use_bias=False)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.blocks = tf.keras.models.Sequential()
        # 构建ResNet网络结构
        for block_id in range(len(block_list)):  # 第几个resnet block
            for layer_id in range(block_list[block_id]):  # 第几个卷积层

                if block_id != 0 and layer_id == 0:  # 对除第一个block以外的每个block的输入进行下采样
                    block = ResnetBlock(self.out_filters, strides=2, residual_path=True)
                else:
                    block = ResnetBlock(self.out_filters, residual_path=False)
                self.blocks.add(block)  # 将构建好的block加入resnet
            self.out_filters *= 2  # 下一个block的卷积核数是上一个block的2倍
        self.p1 = tf.keras.layers.GlobalAveragePooling2D()
        self.f1 = tf.keras.layers.Dense(16, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())

    def call(self, inputs):
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.f1(x)
        return y

class ResnetBlock(Model):

    def __init__(self, filters, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()
        self.filters = filters
        self.strides = strides
        self.residual_path = residual_path

        self.c1 = Conv2D(filters, (3, 3), strides=strides, padding='same', use_bias=False)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')

        self.c2 = Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False)
        self.b2 = BatchNormalization()

        # residual_path为True时，对输入进行下采样，即用1x1的卷积核做卷积操作，保证x能和F(x)维度相同，顺利相加
        if residual_path:
            self.down_c1 = Conv2D(filters, (1, 1), strides=strides, padding='same', use_bias=False)
            self.down_b1 = BatchNormalization()

        self.a2 = Activation('relu')

    def call(self, inputs):
        residual = inputs  # residual等于输入值本身，即residual=x
        # 将输入通过卷积、BN层、激活层，计算F(x)
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)

        x = self.c2(x)
        y = self.b2(x)

        if self.residual_path:
            residual = self.down_c1(inputs)
            residual = self.down_b1(residual)

        out = self.a2(y + residual)  # 最后输出的是两部分的和，即F(x)+x或F(x)+Wx,再过激活函数
        return out
    
class Baseline(Model):
    def __init__(self):
        super(Baseline, self).__init__()
        self.c1 = Conv2D(filters=32, kernel_size=(3, 3), padding='same')  # 卷积层
        self.b1 = BatchNormalization()
        self.a1 = Activation("relu")
        self.p1 = MaxPool2D(pool_size=(2, 2))  # 池化层
        
        self.c2 = Conv2D(filters=64, kernel_size=(3, 3), padding='same')  # 卷积层
        self.b2 = BatchNormalization()
        self.a2 = Activation("relu")
        self.p2 = MaxPool2D(pool_size=(2, 2))  # 池化层
        
        self.c3 = Conv2D(filters=128, kernel_size=(3, 3), padding='same')  # 卷积层
        self.b3 = BatchNormalization()
        self.a3 = Activation("relu")
        self.p3 = MaxPool2D(pool_size=(2, 2))  # 池化层
        
        self.f1 = Flatten()
        self.f2 = Dense(128, kernel_regularizer=tf.keras.regularizers.l2())
        self.b4 = BatchNormalization()
        self.a4 = Activation("relu")
        self.d1 = Dropout(0.2)
        self.f3 = Dense(32, kernel_regularizer=tf.keras.regularizers.l2()) 
        self.b5 = BatchNormalization()
        self.a5 = Activation("relu")
        self.d2 = Dropout(0.2)
        self.f4 = Dense(16, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())
        
    def call(self, x):
        # 卷积层1
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)
        # 卷积层2
        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p2(x)
        # 卷积层3
        x = self.c3(x)
        x = self.b3(x)
        x = self.a3(x)
        x = self.p3(x)
        # 全连接层
        x = self.f1(x)
        x = self.f2(x)
        x = self.b4(x)
        x = self.a4(x)
        x = self.d1(x)
        x = self.f3(x)
        x = self.b5(x)
        x = self.a5(x)
        x = self.d2(x)
        y = self.f4(x)
        return y
    
class NewModel(Model):
    def __init__(self):
        super(NewModel, self).__init__()
        self.c11 = Conv2D(filters=32, kernel_size=(3, 3), padding='same')  # 卷积层
        self.c12 = Conv2D(filters=32, kernel_size=(3, 3), padding='same')  # 卷积层
        self.c13 = Conv2D(filters=32, kernel_size=(3, 3), padding='same')  # 卷积层
        self.b1 = BatchNormalization()
        self.a1 = Activation("relu")
        self.p1 = MaxPool2D(pool_size=(2, 2))  # 池化层
        self.blocks = tf.keras.models.Sequential()
        self.block11 = ResnetBlock(filters=64, strides=2, residual_path=True)
        self.block12 = ResnetBlock(filters=64)
        self.block21 = ResnetBlock(filters=128, strides=2, residual_path=True)
        self.block22 = ResnetBlock(filters=128)        
        self.block31 = ResnetBlock(filters=256, strides=2, residual_path=True)
        self.block32 = ResnetBlock(filters=256, strides=2, residual_path=True)
        self.block33 = ResnetBlock(filters=256)
        self.blocks.add(self.block11)
        self.blocks.add(self.block12)
        self.blocks.add(self.block21)
        self.blocks.add(self.block22)        
        self.blocks.add(self.block31)
        self.blocks.add(self.block32)
        self.blocks.add(self.block33)
        
        self.f1 = Flatten()
        self.f2 = Dense(128, kernel_regularizer=tf.keras.regularizers.l2())
        self.b2 = BatchNormalization()
        self.a2 = Activation("relu")
        self.d1 = Dropout(0.2)
        self.f3 = Dense(32, kernel_regularizer=tf.keras.regularizers.l2()) 
        self.b3 = BatchNormalization()
        self.a3 = Activation("relu")
        self.d2 = Dropout(0.2)
        self.f4 = Dense(16, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())
        
    def call(self, x):
        # 卷积层1
        x = self.c11(x)
        x = self.c12(x)
        x = self.c13(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)
        # resBlock
        x = self.blocks(x)   
        # 全连接层
        x = self.f1(x)
        x = self.f2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.d1(x)
        x = self.f3(x)
        x = self.b3(x)
        x = self.a3(x)
        x = self.d2(x)
        x = self.d1(x)
        y = self.f4(x)
        return y

In [ ]:
def show(history, model_name):
    figure_save_path = "./checkpoint/flower_" + model_name + ".png"
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['font.family'] = ['Times New Roman']
    plt.rcParams['axes.unicode_minus'] = False

    acc = history['sparse_categorical_accuracy']
    val_acc = history['val_sparse_categorical_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    # print(history)
    # 测试集结果
    x = list(history["loss_test"].keys())
    loss_test = list(history["loss_test"].values())
    acc_test = list(history["acc_test"].values())
    print("lost_test", loss_test)
    print("acc_test", acc_test)
    epochs = len(history['loss'])
    
    # 作图：训练集和验证集准确率曲线图，测试集最终结果点
    plt.figure(figsize=(12, 4)) # 画布大小12×3英寸
    plt.subplot(1, 2, 1) # 子图1用于记录准确率
    plt.plot(np.arange(1, epochs + 1), acc, label='train accuracy') # 绘制训练集准确率
    plt.plot(np.arange(1, epochs + 1), val_acc, label='validation accuracy') # 绘制验证集准确率
    plt.scatter(x, acc_test, marker="o", color="red") # 绘制测试集准确率散点
    # for i, x_i in enumerate(x):
    #     plt.plot([x_i, x_i], [0, acc_test[i]], linestyle="--")
    #     plt.plot([0, x_i], [acc_test[i], acc_test[i]], linestyle="--")
    for i, x_i in enumerate(x):  # 循环为测试集散点注解
        plt.annotate("acc=%.3f"% acc_test[i], # 注解文字
                     xytext=(-40, -30), # 注解文字偏移
                     xycoords="data",
                     xy=(x_i, acc_test[i]), # 注解坐标
                     textcoords='offset points', # 注解方式
                     arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2")) # 注解箭头
    plt.xlabel("epochs") # x轴标签
    plt.title('classification accuracy') # y轴标签
    plt.legend() # 显示图注

    plt.subplot(1, 2, 2) # 子图2用于记录损失函数值
    plt.plot(np.arange(1, epochs + 1), loss, label='train loss') # 绘制训练集损失函数值
    plt.plot(np.arange(1, epochs + 1), val_loss, label='validation loss') # 绘制验证集损失函数之
    plt.scatter(x, loss_test, marker="o", color="red") # 绘制测试集损失函数值散点
    for i, x_i in enumerate(x):  # 循环为测试集散点注解
        plt.annotate("loss=%.3f"% loss_test[i],
                     xytext=(-50, 20),
                     xycoords="data",
                     xy=(x_i, loss_test[i]),
                     textcoords='offset points',
                     arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))
    plt.xlabel("epochs")
    plt.title('loss function value')
    plt.legend()

    plt.savefig(figure_save_path) # 存储图像
    plt.show() # 显示图像

def history_show(model_name):
    history_save_path = "./checkpoint/flower_" + model_name + "_history.pickle"
    history = {"sparse_categorical_accuracy": [],
               "val_sparse_categorical_accuracy": [],
               "loss": [],
               "val_loss": [],
               "loss_test": {},
               "acc_test": {},
               "batch_size": [],
               "lr": []
               }
    count = []
    if os.path.exists(history_save_path):
        with open(history_save_path, 'rb') as file:
            all_epochs = 0 # 记录历史轮次
            while True:
                # pickle.dump()以mode="ab"模式下追加的为多个不同的pickle对象
                # 因此需要多次dump将其多个对象加载出来
                try:
                    history_load = pickle.load(file) # 加载多次的历史记录
                    # 本记录若干轮次训练集和验证集acc,loss的追加
                    history["sparse_categorical_accuracy"].extend(history_load["sparse_categorical_accuracy"])
                    history["val_sparse_categorical_accuracy"].extend(history_load["val_sparse_categorical_accuracy"])
                    history["loss"].extend(history_load["loss"])
                    history["val_loss"].extend(history_load["val_loss"])
                    this_epoch = len(history_load["loss"]) # 记录本记录的轮次
                    count.append(this_epoch)
                    all_epochs += this_epoch
                    # 本记录测试集acc,loss的键值对的追加
                    history["loss_test"][all_epochs] = history_load["loss_test"]
                    history["acc_test"][all_epochs] = history_load["acc_test"]
                    # 本记录批量大小和学习率的追加
                    history["batch_size"].append(history_load["batch_size"])
                    history["lr"].append(history_load["lr"])
                except EOFError:
                    train_count = len(count)
                    print("总共进行" + str(train_count) + "次训练：")
                    for i in range(train_count):
                        print("""第{}次训练：\n\
                                 \t训练轮数为：{}轮\n\
                                 \t批量大小为：{}张图片\n\
                                 \t学习率为：{:g}\n
                              """.format(i + 1, count[i], history["batch_size"][i], history["lr"][i]))
                    print("总共训练轮数：%d轮\n" % all_epochs)
                    break
    show(history, model_name)

In [ ]:
def model_fit_data_augment(model, epochs, x_train, y_train, x_test, y_test, x_validation, y_validation, batch_size=32, lr=0.001):
    # 带数据增强的模型训练
    model_name = model.__class__.__name__
    checkpoint_save_path = "./checkpoint/flower_" + model_name + ".ckpt"
    history_save_path = "./checkpoint/flower_" + model_name + "_history.pickle"

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['sparse_categorical_accuracy'])


    if os.path.exists(checkpoint_save_path + '.index'):
        print('-------------load the model-----------------')
        model.load_weights(checkpoint_save_path)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                     save_weights_only=True,
                                                     save_best_only=True)
    # 给训练集添加随机噪声
    image_gen_train = ImageDataGenerator(
        rescale=1. / 1.,  # 如为图像，分母为255时，可归至0～1
        rotation_range=45,  # 随机45度旋转
        width_shift_range=.10,  # 宽度偏移
        height_shift_range=.10,  # 高度偏移
        horizontal_flip=True,  # 水平翻转
        zoom_range=0.2  # 将图像随机缩放阈量20％
    )
    image_gen_train.fit(x_train)

    history = model.fit(image_gen_train.flow(x_train, y_train, batch_size=batch_size),
                        # x_train, y_train, batch_size=32,
                        epochs=epochs,
                        verbose=1,
                        # validation_split=0.1,
                        validation_data=(x_validation, y_validation),
                        validation_freq=1,
                        callbacks=[cp_callback])
    res = model.evaluate(x_test, y_test, verbose=1)
    history.history["loss_test"] = res[0]
    history.history["acc_test"] = res[1]
    history.history["batch_size"] = batch_size
    history.history["lr"] = lr
    model.summary()

    with open(history_save_path, 'ab') as file_pi:
        pickle.dump(history.history, file_pi)

    # show(history.history, model_name)

    
def model_predict(model):
    # 对传入的模型的历史数据作图，并且进行预测
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['sparse_categorical_accuracy'])

    model_name = model.__class__.__name__
    checkpoint_save_path = "./checkpoint/flower_" + model_name + ".ckpt"
    if os.path.exists(checkpoint_save_path + '.index'):
        print('-------------load the model-----------------')
        model.load_weights(checkpoint_save_path)
    else:
        print("No model is chosen")
        exit(0)

    history_show(model_name)

    preNum = int(input("需要预测的花朵图片个数："))

    for i in range(preNum):
        flower = input("\n类别:")
        flower_path = root_path + flower + "/"
        flower_count = 0
        for file in os.listdir(flower_path):
            flower_count += 1
        print(flower + "类花朵共有%d张图片" % flower_count)
        number = input("序号(1~%d)：" % flower_count)
        image_path = flower_path + flower + " (%s).jpg" % number
        img = Image.open(image_path)
        # 展示图片
        image = plt.imread(image_path)
        plt.subplot(1, preNum, i + 1)
        plt.set_cmap('gray')
        plt.imshow(image)
        plt.axis("off")
        img = img.convert("RGB")
        img = img.resize((224, 224), Image.ANTIALIAS)
        img_arr = np.array(img)
        img_arr = img_arr / 255.0
        x = np.reshape(img_arr, (1, 224, 224, 3))
        result = model.predict(x)
        pred = tf.argmax(result, axis=1)
        plt.title(flowers[int(pred.numpy())])
    plt.suptitle("The model is " + model_name)
    plt.tight_layout()
    plt.show()

In [ ]:
model = AlexNet8()
model_fit_data_augment(model, 100,
         x_train, y_train, x_test, y_test, x_validation, y_validation,
         batch_size=64, lr=0.01)

In [ ]:
model_predict(model)

In [ ]:
model.evaluate(x_train, y_train)
model.evaluate(x_validation, y_validation)
model.evaluate(x_test, y_test)

In [ ]:
import os
import zipfile
import datetime
from IPython.display import FileLink

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        print("output.zip has already existed.")
        flag = input("Remove it or not? (Y/N)")
        if flag == "Y":
            print("The old zip is removing...")
            os.remove(zipPath)
            print("The new zip is packing...")
            file2zip(packagePath, zipPath)
            print("Successfully!")
    else:
        print("The zip is packing...")
        file2zip(packagePath, zipPath)
        print("Successfully!")
    print(datetime.datetime.utcnow())
    

In [ ]:
FileLink("output.zip")

In [ ]:
# import shutil
# import os

# if __name__ == '__main__':
#     path = '/kaggle/working'
#     if os.path.exists(path):
#         shutil.rmtree(path)
#         print('删除完成')
#     else:
#         print('原本为空')